v3: SVM
    

In [33]:
# time = 0
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge as KRR
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
import xgboost as xgb

In [34]:
# Data Loading
print("Preparing training files....")

songs_df=pd.read_csv('songs.csv')
df_test=pd.read_csv('test.csv') # Large 500MB
df_submission = pd.read_csv('dummy_submission.csv')
df_new_train = pd.read_csv('train.csv')
song_labels = pd.read_csv('song_labels.csv')
df_train_full=df_new_train.reset_index(drop=True)
train_df = df_train_full

train = train_df
test = df_test
test_df= df_test


Preparing training files....


In [35]:
language_no = len(songs_df['language'].unique())
song_labels = pd.merge(song_labels, songs_df, on='platform_id', how='outer')
song_labels = song_labels.drop(['released_year', 'language', 'number_of_comments'], axis=1)
song_labels.head()


,platform_id,label_id,count,song_id
0,Q207781,10304,6,4785.0
1,Q207781,11973,2,4785.0
2,Q207781,20676,1,4785.0
3,Q207781,23471,17,4785.0
4,Q207781,3042,3,4785.0


In [36]:
song_labels = song_labels.drop_duplicates(subset=['label_id', 'song_id'], keep='last')
song_labels = pd.merge(train, song_labels, on='song_id', how='outer')
song_labels = song_labels.drop(['customer_id','score'], axis=1)
song_labels = song_labels.dropna(subset=['song_id'])
song_labels = song_labels.drop_duplicates(subset=['label_id', 'song_id'], keep='last')
song_labels['song_id'] = song_labels['song_id'].astype(int)
song_labels.head()

,song_id,platform_id,label_id,count
4200,3459,P49540,21815.0,24.0
4201,3459,P49540,14527.0,12.0
4202,3459,P49540,10932.0,14.0
4203,3459,P49540,14552.0,21.0
4204,3459,P49540,20849.0,22.0


In [28]:
df_submission.to_csv('song_lables_edited.csv',index=False)

In [37]:
songs_no = len(train['song_id'].unique())
features = len(song_labels['label_id'].unique())

In [38]:
user_ids = train.customer_id.unique()
user_ids = list(user_ids)
total_users = len(user_ids)

In [41]:
song_labels.groupby(['label_id']).count().mean()

platform_id    99.8004
label_id       99.8004
count          99.8004
dtype: float64

In [18]:
X = song_labels.pivot(index='song_id', columns='label_id', values='count')
X = np.array(X.drop(X.columns[0], axis=1))
X  = np.nan_to_num(X, nan = 0)

pca = PCA(n_components=100)
X = pca.fit_transform(X)

for userId in user_ids:
    User_specific = train_df.loc[train_df['customer_id'] == userId]
    User_specific_test = test_df.loc[test_df['customer_id'] == userId]
    if (len(User_specific)!=0) and (len(User_specific_test)!=0):
        X_training_matrix = X[User_specific.song_id -1,:]
        Y_training_matrix = User_specific.score
        X_testing_matrix = X[User_specific_test.song_id -1,:]
        list_songId = User_specific_test.song_id
        model =  KRR(kernel='rbf')
        model.fit(X_training_matrix, Y_training_matrix)
        Y_test_pred_matrix = model.predict(X_testing_matrix)
        test_df.loc[test_df['customer_id'] == userId,'score']  = Y_test_pred_matrix

In [19]:
df_submission.score = test_df.score

In [20]:
df_submission.loc[df_submission['score']>5,'score'] = 5
df_submission.loc[df_submission['score']<1,'score'] = 1

In [21]:
df_submission.to_csv('./UR_v3.csv',index=False)

In [22]:
df_submission

,test_row_id,score
0,0,1.0
1,1,1.0
2,2,1.0
3,3,1.0
4,4,1.0
...,...,...
670676,670676,1.0
670677,670677,1.0
670678,670678,1.0
670679,670679,1.0


In [23]:
test_df.score

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
670676    0.0
670677    0.0
670678    0.0
670679    0.0
670680    0.0
Name: score, Length: 670681, dtype: float64

In [18]:
def generate_XY():
    X=np.zeros((songs_no,features)) 

    songs_with_featvecs=set(song_labels['song_id'])
    rating_songs = -1*np.ones(songs_no) 

    X = np.array(song_labels.pivot(index='song_id', columns='label_id', values='count'))
    X  = np.nan_to_num(X, nan = -1)

    for i in range(songs_no):
        temp = train_df[train_df['song_id']==i]
        if len(temp)==0:
            continue
        ratings_curr_songs = temp['score']
        rating_songs[i] = np.mean(ratings_curr_songs)


    all_genres = []
    for i in range(songs_no):
        temp = songs_df[songs_df['song_id']==i]
        if len(temp)==0:
            continue
        temp = temp['language'].values[0]
        all_genres.append(temp)
        
#     print(all_genres)

#     X_genre = np.zeros((songs_no,language_no))

#     for i in range(songs_no):
#         temp = songs_df[songs_df['song_id']==i]
#         if len(temp)==0:
#             continue
#         temp = temp['language'].values[0]

#         for idx, genre in enumerate(all_genres):
#             X_genre[i,idx] = genre in temp

#     X_concat = np.concatenate((X,X_genre),axis=1)
    return X, rating_songs

def SVR_Predictions(X, rating_songs):
    X_all = X[rating_songs>0]
    Y_all = rating_songs[rating_songs>0]

    best_kernel_param = 0.1
    best_reg_param = 10

    SVM_algo   = SVR(C=best_reg_param, kernel='rbf', gamma = best_kernel_param, verbose = True)
    classifier = SVM_algo.fit(X_all,Y_all)

    X_all_full = X
    Y_pred_all = classifier.predict(X_all_full)
    return Y_pred_all


X, rating_movies = generate_XY()
print('done generation of X')
Y_pred_all = SVR_Predictions(X, rating_movies)
print('done SVR Predictions')


done generation of X
[LibSVM]done SVR Predictions


In [21]:
Y_pred_all.shape

(10000,)

In [33]:
user_ids.index('J24644')

3074

In [37]:
### USER BASED REGRESSION ###
def userbased_regression():
    kernel_param = 0.1
    C = 10
    alpha = 1/(2*C)

    #For user specific
    rating_pred = np.zeros((songs_no,songs_no))

    for userId in user_ids:
        User_specific = train_df.loc[train_df['customer_id'] == userId]
        User_specific_test = test_df.loc[test_df['customer_id'] == userId]
        if (len(User_specific)!=0) and (len(User_specific_test)!=0):
            X_training_matrix = X[User_specific.song_id -1,:]
            Y_training_matrix = User_specific.score
            X_testing_matrix = X[User_specific_test.song_id -1,:]
            list_songId = User_specific_test.song_id
            SVM_algo =  KRR(kernel='rbf')
            classifier = SVM_algo.fit(X_training_matrix,Y_training_matrix)
            Y_test_pred_matrix = classifier.predict(X_testing_matrix)
            rating_pred[user_ids.index(userId),list_songId.values-1] = Y_test_pred_matrix
    return rating_pred


rating_pred = userbased_regression()

IndexError: index 10000 is out of bounds for axis 0 with size 10000

In [25]:
rating_pred.shape

(10000, 10000)

In [26]:
userid =  df_test.iloc[i,0]

In [27]:
userid

'J24644'

In [ ]:
rating_pred

In [ ]:
# rectifying zero values of user regression values
user_regression = np.zeros(len(df_test))
for i in range(len(df_test)):
    userid =  df_test.iloc[i,0]
    movieid = df_test.iloc[i,1]
    #movie_based
    rating_movie = Y_pred_all[movieid]
    #user based
    rating_user = rating_pred[user_ids.index(userid),movieid-1]
    if rating_user==0:
        rating_user = rating_movie
    user_regression[i] = rating_user


In [6]:
### TRUNCSVD ###

#Removing duplicates
song_matrix = pd.concat([train,test]).drop_duplicates(subset = ['customer_id','song_id'],keep = 'first')
#Creates a song matrix of #numofusers vs #noofsongs
song_matrix = song_matrix.pivot('customer_id','song_id','score')

song_means = song_matrix.mean()
user_means = song_matrix.mean(axis=1)
#Mean shifting
song_shifted_temp = song_matrix-song_means
song_shifted = song_shifted_temp.fillna(0)
#To get locations where we have ratings
mask = -song_shifted_temp.isnull()

def repeated_matrix_reconstruction(num_pcs,num_iterations):
    global song_shifted
    for i in range(num_iterations):
        SVD = TruncatedSVD(n_components=num_pcs,random_state=42)
        SVD.fit(song_shifted)
        #For the ease of applying masks we work with pandas
        song_represented =  pd.DataFrame(SVD.inverse_transform(SVD.transform(song_shifted)),columns=song_shifted.columns,index=song_shifted.index)
        loss = mean_squared_error(song_represented[mask].fillna(0),song_shifted_temp[mask].fillna(0))
        print('Iteration: {} , Loss: {} '.format(i,loss))
        #To just update the non-zero values of song_reprented values to the true ratings
        song_represented[mask] = song_shifted_temp[mask]
        song_shifted = song_represented
    #Mean shifting it back
    song_mat = song_shifted + song_means
    song_mat = song_mat.clip(lower=1,upper=5)
    return song_mat
print("Starting truncated svd with number of components as 20")
representative_matrix_20 = repeated_matrix_reconstruction(20,10)
print("Done")
print("Starting truncated svd with number of components as 15")
representative_matrix_15 = repeated_matrix_reconstruction(15,10)
print("Done")
#bagging
rating_matrix = (representative_matrix_15+representative_matrix_20)/2

Starting truncated svd with number of components as 20
Iteration: 0 , Loss: 0.004090392101434116 
Iteration: 1 , Loss: 0.0039244186879267015 
Iteration: 2 , Loss: 0.0037922913836162535 
Iteration: 3 , Loss: 0.0036813374917077026 
Iteration: 4 , Loss: 0.003585387402402682 
Iteration: 5 , Loss: 0.0035008208988705674 
Iteration: 6 , Loss: 0.0034252374367895392 
Iteration: 7 , Loss: 0.003356930466579425 
Iteration: 8 , Loss: 0.0032946362492698 
Iteration: 9 , Loss: 0.0032373930924846895 
Done
Starting truncated svd with number of components as 15
Iteration: 0 , Loss: 0.0033889477928180562 
Iteration: 1 , Loss: 0.003331020945049362 
Iteration: 2 , Loss: 0.0032813019992975656 
Iteration: 3 , Loss: 0.003237572162961393 
Iteration: 4 , Loss: 0.0031982486506286767 
Iteration: 5 , Loss: 0.0031622759794251885 
Iteration: 6 , Loss: 0.00312895677409973 
Iteration: 7 , Loss: 0.003097815847069581 
Iteration: 8 , Loss: 0.0030685146444273553 
Iteration: 9 , Loss: 0.0030408012373814284 
Done


In [7]:
trunc_prediction = np.zeros(len(test))
for i in range(len(test)):
    userid =  test.iloc[i,0]
    songid = test.iloc[i,1]
    trunc_prediction[i] = rating_matrix[rating_matrix.index==userid][songid].values[0]
    


In [8]:
trunc_prediction

array([3.4163984 , 3.66428208, 4.03433837, ..., 3.74826155, 3.91394637,
       3.23143152])

In [9]:
np.around(trunc_prediction,2)

array([3.42, 3.66, 4.03, ..., 3.75, 3.91, 3.23])

In [ ]:
# ENSEMBLING

PRED = user_regression
PRED = np.around(PRED,1)

PRED = np.clip(PRED, a_min# SUBMISSION
 = 1, a_max = 5)

df_submission.score = PRED
df_submission.to_csv('./UR_v1.csv',index=False)

In [11]:
df_submission.head()

,test_row_id,score
0,0,3.4
1,1,3.7
2,2,4.0
3,3,4.0
4,4,3.9
